# Imports

In [1]:
import pandas as pd
import numpy as np

# Functions

In [300]:
def get_data (path):
    df = pd.read_csv(path)
    df.head()
    df.rename(columns = {'price':'buying_price'}, inplace = True)
    return df

def col_price_median(data):
    df = data[['buying_price', 'zipcode']].groupby('zipcode').median().reset_index()
    df.columns = df.columns.str.replace('buying_price', 'price_median')
    df2 = pd.merge(data,df,on='zipcode',how='inner')
    return df2

def col_status(df2):
    for i in range( len(df2) ):
        if (df2.loc[i, 'buying_price'] < df2.loc[i, 'price_median']) & (df2.loc[i, 'condition'] >= 2):
            df2.loc[i, 'status'] = 'buy'
        else:
            df2.loc[i, 'status'] = 'not_buy'
    return df2

def col_season(data):
    data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
    
    data['season'] = data['date'].apply(lambda x: 
                                        'winter' 
                                        if ts('2014-05-02') <= x <= ts('2014-05-31') else
                                       'summer' 
                                        if ts('2014-06-01') <= x <= ts('2014-11-30') else
                                       'winter')
    return data

def ts(obj):
    return pd.to_datetime(obj)

def col_season_median(data):
    df = data[['season', 'zipcode', 'buying_price']].groupby(['season', 'zipcode']).median()
    df.columns = df.columns.str.replace('buying_price', 'season_median')
    df2 = pd.merge(data,df,on=['zipcode', 'season'],how='inner')
    return df2

def col_selling_price(data):
    data['selling_price'] = float(0)
    for i in range( len(data) ):
        if (data.loc[i, 'buying_price'] < data.loc[i, 'season_median']) & (data.loc[i, 'status'] == 'buy'):
            data.loc[i, 'selling_price'] = float(data.loc[i, 'buying_price']) * 1.30
        if (data.loc[i, 'buying_price'] >= data.loc[i, 'season_median']) & (data.loc[i, 'status'] == 'buy'):
            data.loc[i, 'selling_price'] = float(data.loc[i, 'buying_price']) * 1.10
    return data

def col_profit(data):
    data['profit'] = float(0)
    for i in range(len(data)):
        if data.loc[i, 'selling_price'] != 0:
            data.loc[i, 'profit'] = float(data.loc[i, 'selling_price']) - float(data.loc[i, 'buying_price'])
        else:
            None
    return data

## Data Extraction

In [288]:
path = 'kc_house_data.csv'
data = get_data(path)

## Data Visualization

In [248]:
data.columns

Index(['id', 'date', 'buying_price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [294]:
data

,id,date,buying_price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,zipcode,lat,long,sqft_living15,sqft_lot15,price_median,status,season,season_median,selling_price
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,...,98178,47.5112,-122.257,1340,5650,278277.0,buy,summer,271000.0,288470.0
1,4060000240,2014-06-23,205425.0,2,1.00,880,6780,1.0,0,0,...,98178,47.5009,-122.248,1190,6780,278277.0,buy,summer,271000.0,267052.5
2,4058801670,2014-07-17,445000.0,3,2.25,2100,8201,1.0,0,2,...,98178,47.5091,-122.244,2660,8712,278277.0,not_buy,summer,271000.0,0.0
3,2976800796,2014-09-25,236000.0,3,1.00,1300,5898,1.0,0,0,...,98178,47.5053,-122.255,1320,7619,278277.0,buy,summer,271000.0,306800.0
4,3969300030,2014-07-23,165000.0,4,1.00,1000,7134,1.0,0,0,...,98178,47.4897,-122.240,1020,7138,278277.0,buy,summer,271000.0,214500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,7397300170,2014-05-30,3710000.0,4,3.50,5550,28078,2.0,0,2,...,98039,47.6395,-122.234,2980,19602,1892500.0,not_buy,winter,1832500.0,0.0
21609,3262300920,2015-04-08,1200000.0,4,3.00,2150,8119,2.0,0,0,...,98039,47.6335,-122.236,1590,8119,1892500.0,buy,winter,1832500.0,1560000.0
21610,2470200020,2014-05-14,1880000.0,4,2.75,3260,19542,1.0,0,0,...,98039,47.6245,-122.236,3480,19863,1892500.0,buy,winter,1832500.0,1692000.0
21611,3262300485,2015-04-21,2250000.0,5,5.25,3410,8118,2.0,0,0,...,98039,47.6295,-122.236,3410,16236,1892500.0,not_buy,winter,1832500.0,0.0


In [279]:
data.dtypes

id                        int64
date             datetime64[ns]
buying_price            float64
bedrooms                  int64
bathrooms               float64
sqft_living               int64
sqft_lot                  int64
floors                  float64
waterfront                int64
view                      int64
condition                 int64
grade                     int64
sqft_above                int64
sqft_basement             int64
yr_built                  int64
yr_renovated              int64
zipcode                   int64
lat                     float64
long                    float64
sqft_living15             int64
sqft_lot15                int64
price_median            float64
status                   object
season                   object
season_median           float64
selling_price           float64
dtype: object

## Data Transformation

In [289]:
# setting the price_median column
data = col_price_median(data)

In [290]:
# setting the status column - ready for the buying report
data = col_status(data)

In [291]:
# setting the seson column
data = col_season(data)

In [292]:
# setting the season_median column - price median by season and zipcode 
data = col_season_median(data)

In [297]:
# setting the selling_price column - ready for the selling report
data = col_selling_price(data)

In [301]:
# setting the profit column
data = col_profit(data)

In [309]:
report = data[['id', 'zipcode','season', 'price_median', 'buying_price','status', 'selling_price', 'profit']]
report.to_csv('buying-selling-report.csv',index=False) 

In [310]:
report.to_excel('buying-selling-report.xlsx', index=False)